In [4]:
import json

In [5]:
# 每个query底下标注的文档，以及每个query下的bm25top200
labeled_docs = []
LeCaRDv2_label = {}
with open('/share/kelong/chenlong/Legal/data/LeCaRDv2-main/label/relevence.trec', 'r') as f:
    for line in f:
        qid, _, did, rel = line.strip().split('\t')
        labeled_docs.append(did)
labeled_docs = list(set(labeled_docs))

with open('/share/kelong/chenlong/Legal/data/LeCaRDv2-main/label/relevence.trec', 'r') as f:
    for line in f:
        qid, _, did, rel = line.strip().split('\t')
        if qid not in LeCaRDv2_label.keys():
            LeCaRDv2_label[qid] = {}
        LeCaRDv2_label[qid][did] = int(rel)

In [6]:
len(labeled_docs)

21300

In [15]:
generated_docs = json.load(open('/share/kelong/chenlong/Legal/Ranking_LeCaRDv2/examples/LM生成数据/generate_fact/Qwen72B-Chat/doc_fact.json', 'r'))
LeCaRDv2_train_querys = {}
LeCaRDv2_test_querys = {}
with open('/share/kelong/chenlong/Legal/data/LeCaRDv2-main/query/train_query.jsonl', 'r') as f:
    for line in f:
        query = json.loads(line)
        LeCaRDv2_train_querys[str(query['id'])] = query['fact']

with open('/share/kelong/chenlong/Legal/data/LeCaRDv2-main/query/test_query.jsonl', 'r') as f:
    for line in f:
        query = json.loads(line)
        LeCaRDv2_test_querys[str(query['id'])] = query['fact']

In [16]:
LeCaRDv2_bm25 = json.load(open('./bm25_top1k.json', 'r'))
bm25_docs = []
for qid in LeCaRDv2_bm25.keys():
    for did in list(LeCaRDv2_bm25[qid].keys())[100:110]:
        bm25_docs.append(did)
for qid in LeCaRDv2_train_querys.keys():
    if all([LeCaRDv2_label[qid][did] != 3 for did in LeCaRDv2_label[qid].keys()]):
        golden_labels = [2,3]
    else:
        golden_labels = [3]
        
    for did in LeCaRDv2_label[qid].keys():
        negatives = [i for i in LeCaRDv2_bm25[qid].keys() if (i not in LeCaRDv2_label[qid].keys()) or (LeCaRDv2_label[qid][i] not in golden_labels)]
        bm25_docs.extend(negatives[10:10+15])

for qid in LeCaRDv2_test_querys.keys():
    if all([LeCaRDv2_label[qid][did] != 3 for did in LeCaRDv2_label[qid].keys()]):
        golden_labels = [2,3]
    else:
        golden_labels = [3]
        
    if all(LeCaRDv2_label[qid][did] in golden_labels for did in LeCaRDv2_label[qid].keys()):
        extra_candidates = [i for i in LeCaRDv2_bm25[qid].keys() if (i not in LeCaRDv2_label[qid].keys()) or (LeCaRDv2_label[qid][i] not in golden_labels)]
        bm25_docs.extend(extra_candidates[100:110])        
bm25_docs = list(set(bm25_docs))

In [17]:
len(bm25_docs)

12040

In [18]:
len([i for i in bm25_docs if i not in generated_docs.keys()])

1025

In [14]:
len(generated_docs)

28363